# Implementation of Evolutionary Forest
Reference: [Paper](./An_Evolutionary_Forest_for_Regression.pdf)

### Import Libraries and Install them if not installed

#### Install Libraries

In [1]:
%pip install pandas
%pip install numpy
%pip install matplotlib
%pip install seaborn
%pip install git+https://github.com/hengzhe-zhang/EvolutionaryForest.git
%pip install gplearn
%pip install scipy
%pip install hdfe
%pip install deap==1.3.3
%pip install sympy
%pip install scikit_learn
%pip install dill
%pip install lightgbm
%pip install skorch
%pip install umap-learn
%pip install category_encoders
%pip install pyade
%pip install networkx
%pip install torch
%pip install tpot
%pip install linear-tree
%pip install sklearn2pmml
%pip install shap
%pip install mlxtend

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.




  Cloning https://github.com/hengzhe-zhang/EvolutionaryForest.git to c:\users\user\appdata\local\temp\pip-req-build-h9t_cixa
  Resolved https://github.com/hengzhe-zhang/EvolutionaryForest.git to commit 9f04d3838cccfdaa916d4d71ace73a3827eafcc2
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for evolutionary-forest: filename=evolutionary_forest-0.2.2-py2.py3-none-any.whl size=155886 sha256=03c29f25153a28631af53d11a1d5a432601139bf1efc4800feb682065333ac3d
  Stored in directory: C:\Users\user\AppData\Local\Temp\pip-ephem-wheel-cache-f9lvavvr\wheels\57\19\c0\c946b182439b5109b1a5578ec65efac01f28f65e13bb56c4c4
Successfully built evolutionary-forest


  Running command git clone --filter=blob:none --quiet https://github.com/hengzhe-zhang/EvolutionaryForest.git 'C:\Users\user\AppData\Local\Temp\pip-req-build-h9t_cixa'


...

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


#### Import Libraries

In [2]:
import pandas as pd
from evolutionary_forest.forest import EvolutionaryForestRegressor
import sklearn.model_selection as ms
from sklearn.metrics import accuracy_score, r2_score
from sklearn.ensemble import RandomForestRegressor

Failed to initialize NumPy: module compiled against API version 0x10 but this version of numpy is 0xf . Check the section C-API incompatibility at the Troubleshooting ImportError section at https://numpy.org/devdocs/user/troubleshooting-importerror.html#c-api-incompatibility for indications on how to solve this problem . (Triggered internally at ..\torch\csrc\utils\tensor_numpy.cpp:84.)


### Import Dataset

#### Load the Dataset

In [3]:
data = pd.read_csv('./AirQualityUCI.csv', sep=';')

#### Dataset Description

In [4]:
data.head()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9471 entries, 0 to 9470
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           9357 non-null   object 
 1   Time           9357 non-null   object 
 2   CO(GT)         9357 non-null   object 
 3   PT08.S1(CO)    9357 non-null   float64
 4   NMHC(GT)       9357 non-null   float64
 5   C6H6(GT)       9357 non-null   object 
 6   PT08.S2(NMHC)  9357 non-null   float64
 7   NOx(GT)        9357 non-null   float64
 8   PT08.S3(NOx)   9357 non-null   float64
 9   NO2(GT)        9357 non-null   float64
 10  PT08.S4(NO2)   9357 non-null   float64
 11  PT08.S5(O3)    9357 non-null   float64
 12  T              9357 non-null   object 
 13  RH             9357 non-null   object 
 14  AH             9357 non-null   object 
 15  Unnamed: 15    0 non-null      float64
 16  Unnamed: 16    0 non-null      float64
dtypes: float64(10), object(7)
memory usage: 1.2+ MB


### Preprocessing

In [5]:
import pandas as pd

def replace_comma_with_period(value):
    if isinstance(value, str):
        return float(value.replace(',', '.'))
    else:
        return value

# Read the data from the CSV file
data = pd.read_csv('./AirQualityUCI.csv', sep=';')

# Drop unnecessary columns
data.drop(['Date', 'Time', 'Unnamed: 15', 'Unnamed: 16'], axis=1, inplace=True)

# Apply the function to each element in the DataFrame
data = data.applymap(replace_comma_with_period)

# Check for NaN values and drop corresponding rows
data = data.dropna()

# Print the processed data
print(data)


      CO(GT)  PT08.S1(CO)  NMHC(GT)  C6H6(GT)  PT08.S2(NMHC)  NOx(GT)  \
0        2.6       1360.0     150.0      11.9         1046.0    166.0   
1        2.0       1292.0     112.0       9.4          955.0    103.0   
2        2.2       1402.0      88.0       9.0          939.0    131.0   
3        2.2       1376.0      80.0       9.2          948.0    172.0   
4        1.6       1272.0      51.0       6.5          836.0    131.0   
...      ...          ...       ...       ...            ...      ...   
9352     3.1       1314.0    -200.0      13.5         1101.0    472.0   
9353     2.4       1163.0    -200.0      11.4         1027.0    353.0   
9354     2.4       1142.0    -200.0      12.4         1063.0    293.0   
9355     2.1       1003.0    -200.0       9.5          961.0    235.0   
9356     2.2       1071.0    -200.0      11.9         1047.0    265.0   

      PT08.S3(NOx)  NO2(GT)  PT08.S4(NO2)  PT08.S5(O3)     T    RH      AH  
0           1056.0    113.0        1692.0     

### Split the Dataset to Train and Test
Target feature column "T" which is the temperature from Date;Time;CO(GT);PT08.S1(CO);NMHC(GT);C6H6(GT);PT08.S2(NMHC);NOx(GT);PT08.S3(NOx);NO2(GT);PT08.S4(NO2);PT08.S5(O3);T;RH;AH;; in the dataset

In [6]:
x = data.drop(['T'], axis=1)
y = data['T']

x_train, x_test, y_train, y_test = ms.train_test_split(x, y, test_size=0.2, random_state=42)

### Train the model with random forest

In [7]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(x_train, y_train)

y_pred = rf_model.predict(x_test)

# R2
print(r2_score(y_test, y_pred))

0.9999734818663446


### Train the model with evolutionary forest

In [8]:
ef_model = EvolutionaryForestRegressor(max_height=5, normalize=True, select='AutomaticLexicase',gene_num=10, boost_size=100, n_gen=20, n_pop=200, cross_pb=1,base_learner='Random-DT', verbose=True, n_process=1)
ef_model.fit(x_train, y_train)

y_pred = ef_model.predict(x_test)

# R2
print(r2_score(y_test, y_pred))

data shape (7485, 12) (7485,)
   	      	                                                          fitness                                                          	                                  size                                  
   	      	---------------------------------------------------------------------------------------------------------------------------	------------------------------------------------------------------------
gen	nevals	25%         	75%         	avg        	gen	max         	median      	min        	nevals	std        	25%	75%	avg  	gen	max	median	min	nevals	std     
0  	200   	[0.98428388]	[0.99295341]	[0.9887368]	0  	[0.99968164]	[0.98924832]	[0.9742212]	200   	[0.0058378]	3  	4  	3.825	0  	6  	4     	3  	200   	0.635905
defaultdict(<class 'int'>, {'1': 200})
P value of different population 3.878714475901378e-11
Mul(PT08.S2(NMHC), PT08.S4(NO2))
AQ(0, PT08.S4(NO2))
Mul(AQ(NO2(GT), NO2(GT)), AH)
Add(Sub(1, PT08.S4(NO2)), AQ(1, CO(GT)))
AQ(AH, AH)
Mul(RH, 